In [48]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point

In [49]:
data = {'location':['Boulder Colorado','Lancaster, Pa.','Cadair Idris - Penygadair, Dolgellau, UK','Fairbourne, UK','Elan Valley Reservoirs, Rhayader, UK','Snowdonia National Park, Penrhyndeudraeth, UK','Llyswen, Brecon, UK','Black Mountains, Wales, Abergavenny, UK','Kennedy Space Center, Florida'],
       'label':['A place to visit','There\'s good dancing here','Scenic gentle hiking and camping','A secret place that\'s been forgotten','Where you and James & got engaged','What sounds like a wonderful place to visit','Your current home','Beautiful background scenery','A major science center']}

df = pd.DataFrame(data)

In [50]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [51]:
df['loc'] = df['location'].apply(geolocator.geocode, timeout=10)

In [52]:
df['loc']

0       (Boulder, CO, USA, (40.0149856, -105.2705456))
1      (Lancaster, PA, USA, (40.0378755, -76.3055144))
2    (Cadair Idris - Penygadair, Dolgellau LL40 1TL...
3    (Fairbourne LL38 2EJ, UK, (52.696651, -4.054346))
4    (Elan Valley Reservoirs, Rhayader LD6 5HF, UK,...
5    (Snowdonia National Park, Penrhyndeudraeth LL4...
6    (Llyswen, Brecon LD3 0UY, UK, (52.033179999999...
7    (Black Mountains, Wales, Abergavenny NP7 7NP, ...
8    (Florida 32899, USA, (28.5728722, -80.64898079...
Name: loc, dtype: object

In [53]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [54]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,location,label,loc,point,lat,lon,altitude
0,Boulder Colorado,A place to visit,"(Boulder, CO, USA, (40.0149856, -105.2705456))","(40.0149856, -105.2705456, 0.0)",40.014986,-105.270546,0.0
1,"Lancaster, Pa.",There's good dancing here,"(Lancaster, PA, USA, (40.0378755, -76.3055144))","(40.0378755, -76.3055144, 0.0)",40.037875,-76.305514,0.0
2,"Cadair Idris - Penygadair, Dolgellau, UK",Scenic gentle hiking and camping,"(Cadair Idris - Penygadair, Dolgellau LL40 1TL...","(52.6996388, -3.9088055, 0.0)",52.699639,-3.908806,0.0
3,"Fairbourne, UK",A secret place that's been forgotten,"(Fairbourne LL38 2EJ, UK, (52.696651, -4.054346))","(52.696651, -4.054346, 0.0)",52.696651,-4.054346,0.0
4,"Elan Valley Reservoirs, Rhayader, UK",Where you and James & got engaged,"(Elan Valley Reservoirs, Rhayader LD6 5HF, UK,...","(52.272222, -3.688889, 0.0)",52.272222,-3.688889,0.0
5,"Snowdonia National Park, Penrhyndeudraeth, UK",What sounds like a wonderful place to visit,"(Snowdonia National Park, Penrhyndeudraeth LL4...","(53.0932155, -3.801704, 0.0)",53.093215,-3.801704,0.0
6,"Llyswen, Brecon, UK",Your current home,"(Llyswen, Brecon LD3 0UY, UK, (52.033179999999...","(52.03317999999999, -3.2659789, 0.0)",52.033180,-3.265979,0.0
7,"Black Mountains, Wales, Abergavenny, UK",Beautiful background scenery,"(Black Mountains, Wales, Abergavenny NP7 7NP, ...","(51.9666667, -3.1166667, 0.0)",51.966667,-3.116667,0.0
8,"Kennedy Space Center, Florida",A major science center,"(Florida 32899, USA, (28.5728722, -80.64898079...","(28.5728722, -80.64898079999999, 0.0)",28.572872,-80.648981,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [55]:
df.columns

Index(['location', 'label', 'loc', 'point', 'lat', 'lon', 'altitude'], dtype='object')

In [58]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), tiles='Stamen Terrain', zoom_start=2)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Some of the places we\'ve talked about...')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r['label'],r['location'],'---------')
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'{bold_start}{info[1]}{bold_end}{new_line}\
    {bold_start}{info[2]}{bold_end}{new_line}\
    {bold_start}{info[0]}{bold_end}'
    
    folium.Marker(location=location,tooltip=text
                      ).add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))

m

In [60]:
m.save('index.html')